**데이터 불러오기**

In [1]:
import pandas as pd
data = pd.read_csv('./data/000660.csv')

**데이터 칼럼명 영어로 바꾸기**

In [2]:
data.columns = ['Date', 'Close', 'Open', 'High', 'Low', 'Volume', 'Rate']

**데이터 날짜 형식 바꾸기**

In [3]:
Date = data['Date'].apply(lambda x : x.replace('년 ','-'))\
                   .apply(lambda x : x.replace('월 ','-'))\
                   .apply(lambda x : x.replace('일',''))
data['Date'] = Date

**데이터에 들어있는 숫자형식 바꾸기 (, 제거)**

In [4]:
import numpy as np
Close = data['Close'].apply(lambda x : x.replace(',','')).apply(lambda x : np.float(x))
data['Close'] = Close

Open = data['Open'].apply(lambda x : x.replace(',','')).apply(lambda x : np.float(x))
data['Open'] = Open

High = data['High'].apply(lambda x : x.replace(',','')).apply(lambda x : np.float(x))
data['High'] = High

Low = data['Low'].apply(lambda x : x.replace(',','')).apply(lambda x : np.float(x))
data['Low'] = Low

**데이터 순서 뒤집기**

In [5]:
data = data[::-1]

**Index 날짜로 지정하기**

In [6]:
data = data.set_index('Date')

**필요없는 변수(Volumne, Rate) 제거하기**

In [7]:
del data['Volume']
del data['Rate']

**중간 결과**

In [8]:
data

,Close,Open,High,Low
Date,,,,
2011-08-23,17050.0,16200.0,17050.0,16000.0
2011-08-24,16250.0,17200.0,17550.0,15900.0
2011-08-25,17300.0,16900.0,17700.0,16700.0
2011-08-26,17950.0,17050.0,17950.0,16950.0
2011-08-29,19450.0,18550.0,19600.0,18350.0
...,...,...,...,...
2020-02-13,102000.0,100500.0,103500.0,100500.0
2020-02-14,104500.0,101500.0,105000.0,101000.0
2020-02-16,104500.0,104500.0,104500.0,104500.0


**1일 후의 종가 생성**

In [9]:
data['Close_1'] = data['Close'].shift(-1)

**1일 후와 현재 종가 사이의 차이 계산**

In [10]:
data['Label'] = data['Close_1'] - data['Close']

**상승 하락으로 라벨링**

In [11]:
data['UP_DOWN'] = data['Label'].apply(lambda x : 1 if x > 0 else -1)

**필요없는 변수 제거**

In [12]:
del data['Close_1']
del data['Label']

**중간 결과**

In [13]:
data

,Close,Open,High,Low,UP_DOWN
Date,,,,,
2011-08-23,17050.0,16200.0,17050.0,16000.0,-1
2011-08-24,16250.0,17200.0,17550.0,15900.0,1
2011-08-25,17300.0,16900.0,17700.0,16700.0,1
2011-08-26,17950.0,17050.0,17950.0,16950.0,1
2011-08-29,19450.0,18550.0,19600.0,18350.0,-1
...,...,...,...,...,...
2020-02-13,102000.0,100500.0,103500.0,100500.0,1
2020-02-14,104500.0,101500.0,105000.0,101000.0,-1
2020-02-16,104500.0,104500.0,104500.0,104500.0,1


**Train Test Split**

In [14]:
train = data[:2000]
test = data[2001:]

**Target Input Split**

In [15]:
X_train = train[train.columns[:-1]]
y_train = train[train.columns[-1:]]
X_test = test[test.columns[:-1]]
y_test = test[test.columns[-1:]]

**넘파이 어레이로 변경**

In [16]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

**데이터 정규화**

In [17]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std
X_test -= mean
X_test /= std

**DNN**

In [18]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(X_train.shape[-1],)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=512,
                    verbose = 1)

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.8704 - acc: 0.2600
Epoch 2/20
2000/2000 [==============================] - 0s 11us/step - loss: 0.7964 - acc: 0.2545
Epoch 3/20
2000/2000 [==============================] - 0s 11us/step - loss: 0.7461 - acc: 0.2505
Epoch 4/20
2000/2000 [==============================] - 0s 9us/step - loss: 0.7040 - acc: 0.2450
Epoch 5/20
2000/2000 [==============================] - 0s 10us/step - loss: 0.6665 - acc: 0.2395
Epoch 6/20
2000/2000 [==============================] - 0s 10us/step - loss: 0.6319 - acc: 0.2345
Epoch 7/20
2000/2000 [==============================] - 0s 9us/step - loss: 0.5991 - acc: 0.2285
Epoch 8/20
2000/2000 [==============================] - 0s 9us/step - loss: 0.5679 - acc: 0.2195
Epoch 9/20
2000/2000 [==============================] - 0s 9us/step - loss: 0.5381 - acc: 0.2030
Epoch 10/20
2000/2000 [

In [19]:
result = pd.DataFrame({'DNN' : model.predict(X_test).reshape(-1)})
line = np.median(model.predict(X_test).reshape(-1))
result['PRED'] = result['DNN'].apply(lambda x : 1 if x > line else -1)
pred = result['PRED']
test = test['UP_DOWN']
test = test.reset_index()
result = pd.concat([pred, test], axis=1)
result = result[['PRED', 'UP_DOWN']]
result['OX'] = result['UP_DOWN'] - result['PRED']
result['ACC'] = result['OX'].apply(lambda x : 1 if x == 0 else 0)
result

,PRED,UP_DOWN,OX,ACC
0,-1,-1,0,1
1,-1,1,2,0
2,-1,1,2,0
3,-1,1,2,0
4,-1,-1,0,1
...,...,...,...,...
391,-1,1,2,0
392,-1,-1,0,1
393,-1,1,2,0
394,-1,-1,0,1


In [20]:
print('Test Accuracy = {:.2f}%' .format(100 * np.sum(result.ACC) / 396))

Test Accuracy = 51.52%
